<a href="https://colab.research.google.com/github/rohandawar/CarBrandClassificationinDeep-Learning/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link for how to copy kaggle data into google colab:
https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a

another link:

https://pub.towardsai.net/car-brand-classification-in-deep-learning-with-python-d08c54add941

In [1]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

In [3]:
#check the present working directory
!pwd

/content


In [4]:
#change to the directory where the file is present
%cd /content/gdrive/My Drive/Kaggle
!pwd

/content/gdrive/My Drive/Kaggle
/content/gdrive/My Drive/Kaggle


In [5]:
#copy and paste the API command from kaggle to collab notebook
!kaggle datasets download -d ritesh2000/car-brand-images-dataset

  0% 0.00/1.06M [00:00<?, ?B/s]
100% 1.06M/1.06M [00:00<00:00, 35.6MB/s]


In [6]:
#Check the content of the diectory
!ls

car-brand-images-dataset.zip  kaggle.json  Test  Train


In [7]:
#unzip the data and remove the ZIP file
!unzip \*.zip && rm *.zip

Archive:  car-brand-images-dataset.zip
replace Test/audi/21.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Test/audi/22.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Test/audi/22.jpg        
  inflating: Test/audi/23.jpg        
  inflating: Test/audi/24.jpg        
  inflating: Test/audi/25.jpg        
  inflating: Test/audi/26.jpg        
  inflating: Test/audi/27.jpg        
  inflating: Test/audi/28.jpg        
  inflating: Test/audi/29.jpg        
  inflating: Test/lamborghini/1.jpg  
  inflating: Test/lamborghini/10.jpg  
  inflating: Test/lamborghini/11.jpg  
  inflating: Test/lamborghini/12.jpg  
  inflating: Test/lamborghini/13.jpg  
  inflating: Test/lamborghini/14.jpg  
  inflating: Test/lamborghini/15.jpg  
  inflating: Test/lamborghini/16.jpg  
  inflating: Test/lamborghini/17.jpg  
  inflating: Test/lamborghini/18.jpg  
  inflating: Test/lamborghini/19.jpg  
  inflating: Test/lamborghini/2.jpg  
  inflating: Test/lamborghini/20.jog.jpg  
  inflating: Tes

In [8]:
# Importing the important Libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

#import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [9]:
#resizing all the images to 224, 224
IMAGE_SIZE = [224, 224]

train_path = 'Train'
test_path = 'Test'

In [10]:
# Use imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet',
                  include_top = False)

94773248/94765736 [==============================] - 1s 0us/step


In [11]:
#don't train existing weights

for layer in resnet.layers:
  layer.trainable = False

#no of output classes
folders = glob('Train/*')

#our Layers
x = Flatten()(resnet.output)

prediction = Dense(len(folders), activation='softmax')(x)

#creating object model

model = Model(inputs= resnet.input, outputs= prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [12]:
#compile the model

model.compile(loss='categorical_crossentropy', optimizer= 'adam', 
              metrics = ['accuracy'])


#Using the Image Data Generator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('Train', 
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

r = model.fit_generator(training_set, validation_data=test_set,
                        epochs = 50,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
2/2 [==============================] - 25s 16s/step - loss: 3.6558 - accuracy: 0.3333 - val_loss: 8.9972 - val_accuracy: 0.5172
Epoch 2/50
2/2 [==============================] - 19s 14s/step - loss: 12.6883 - accuracy: 0.3229 - val_loss: 6.4754 - val_accuracy: 0.3276
Epoch 3/50
2/2 [==============================] - 19s 14s/step - loss: 7.1456 - accuracy: 0.3958 - val_loss: 4.1077 - val_accuracy: 0.3448
Epoch 4/50
2/2 [==============================] - 19s 14s/step - loss: 3.7652 - accuracy: 0.4792 - val_loss: 10.3613 - val_accuracy: 0.1552
Epoch 5/50
2/2 [==============================] - 19s 14s/step - loss: 7.3906 - accuracy: 0.3021 - val_loss: 2.6609 - val_accuracy: 0.4828
Epoch 6/50
2/2 [==============================] - 19s 14s/step - loss: 2.8868 - accuracy: 0.4271 - val_loss: 2.1922 - val_accuracy: 0.5690
Epoch 7/50
2/2 [==============================] - 19s 14s/step - loss: 3.2179 - accuracy: 0.3958 - val_loss: 4.2084 - val_accuracy: 0.3276
Epoch 8/50
2/2 [=========